<a href="https://colab.research.google.com/github/PexMor/jupyter-tabular-output/blob/main/Better+tabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Better tables in Jupyter notebooks

Purpose of this notebook is to show an option to get beyond the `print` inside Jyputer when producing the tabular content. As the Jupyter is embedded inside HTML and it offers the capability to use HTML for formating among other options then making more visually appealing tables is great option for more legible data formating.

## The code

Let's start with some code hygiene, capture versions and environment so anyone reading this has better starting position.

In [1]:
!pip -q install watermark tabulate

In [2]:
import watermark

%load_ext watermark

Record the versions into the notebook output

In [3]:
%watermark -i -n -v -m -iv

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.25.0

Compiler    : Clang 12.0.5 (clang-1205.0.22.9)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

watermark: 2.2.0



The core of this notebook is the following class `TabOut` which does the table construction over time (i.e. in loop).

In [4]:
from IPython import display
import tabulate
import datetime
class TabOut:
    def __init__(self, headers, show_rel_ts = True, show_abs_ts = True):
        self.headers = headers
        self.show_rel_ts = show_rel_ts
        self.show_abs_ts = show_abs_ts
        if self.show_rel_ts:
            self.headers.append("Rel.time[s]")
        if self.show_abs_ts:
            self.headers.append("Abs.time[s]")
        self.data = []
        self.last_ts = self.base_ts = datetime.datetime.now()
        
    def display(self, row):
        loc_row = row.copy()
        now_ts = datetime.datetime.now()
        dur_rel = (now_ts - self.last_ts).total_seconds()
        dur_abs = (now_ts - self.base_ts).total_seconds()
        self.last_ts = now_ts
        if self.show_rel_ts:
            loc_row.append(dur_rel)
        if self.show_abs_ts:
            loc_row.append(dur_abs)
        self.data.append(loc_row)
        display.clear_output(wait=True)
        display.display(display.HTML(tabulate.tabulate(self.data, tablefmt='html',headers=self.headers)))
    
    def clear(self):
        self.data = []
        
    def done(self):
        display.clear_output(wait=True)
        html = tabulate.tabulate(self.data, tablefmt='html',headers=self.headers)
        html += "<h2>Done!</h2>"
        display.display(display.HTML(html))

## Demo time

In [5]:
import time
table = [["Sun",696000,1989100000],
         ["Earth",6371,5973.6],
         ["Moon",1737,73.5],
         ["Mars",3390,641.85]]
ota = []
out = TabOut(headers=["Planet","R (km)", "mass (x 10^29 kg)"])
for no in range(0,len(table)):
    out.display(table[no])
    time.sleep(3)
out.done()

Planet,R (km),mass (x 10^29 kg),Rel.time[s],Abs.time[s]
Sun,696000,1.9891e+09,6.3e-05,6.3e-05
Earth,6371,5973.6,3.01531,3.01537
Moon,1737,73.5,3.00775,6.02312
Mars,3390,641.85,3.00799,9.03111
